In [1]:
import os
import sys
import pandas as pd
from Bio import SeqIO
from Bio import Seq
import pysam
import re

In [2]:
#change to working directory 
os.chdir('/mnt/d/rDNA/')

In [102]:
#range is optimized for 5S rDNAs
def ReadCigar2df(start, cigar_string, bin_set = 2):
    global indel_df
    for num , IDM in re.findall('(\d+)([IDM])', cigar_string):
        if IDM == 'M':
            start += int(num)
            continue
        #print(num, IDM, start)
        if IDM == "D":
            if int(num) > int(bin_set):
                for i in range( start ,start + int(num)):
                    indel_df.DEL[i] += 1
            start += int(num)
        if IDM == "I":
            if int(num) > int(bin_set):
                for i in range( start ,start + int(num)):
                    if i < 1000:#prevent end exceed of end insertions from supplementary alignmetn cigar
                        indel_df.INS[i] += 1
#range is optimized for 45S rDNAs
def Read_45SrDNA_Cigar2df(start, cigar_string, bin_set = 2):
    global indel_df
    for num , IDM in re.findall('(\d+)([IDM])', cigar_string):
        if IDM == 'M':
            start += int(num)
            continue
        #print(num, IDM, start)
        if IDM == "D":
            if int(num) > int(bin_set):
                for i in range( start ,start + int(num)):
                    indel_df.DEL[i] += 1
            start += int(num)
        if IDM == "I":
            if int(num) > int(bin_set):
                for i in range( start ,start + int(num)):
                    indel_df.INS[i] += 1

In [7]:
#5S
stage = "N2YA"
rDNA = "5S"
fname = "sam/" + stage + "_" + rDNA + ".sam" 
set_bin = [2,5,10]
for bin_set in set_bin:
    counter = 0
    #make empty df to load indel with bin size 
    indel_df = pd.DataFrame({"pos": list(range(1,1001))})
    indel_df["DEL"] = [0]*1000
    indel_df["INS"] = [0]*1000
    indel_fname =  "indel_df/" + stage + "_" + rDNA + "_bin_" + str(bin_set) + ".tsv" 
    with open(fname , "r") as samfile:
        for _ in range(4):
            next(samfile)
        for line in samfile:    
            sam_content = line.split("\t")
            flag = sam_content[1]
            if flag != 4 :
                counter += 1
                pos = int(sam_content[3])
                cigar = str(sam_content[5])
                ReadCigar2df(pos, cigar , bin_set)
                sys.stdout.write('\r'+"{} cigar processed".format(counter))
        indel_df.to_csv(indel_fname, index=False, header=True, sep="\t")
    print("\nExtracted {} records of cigar to file with minimal INDEL size {}\n".format(counter , bin_set))

NameError: name 'ReadCigar2df' is not defined

In [12]:
stage = "AF16"
rDNA = "45S"
fname = "sam/" + stage + "_" + rDNA + ".sam" 
set_bin = [2,5,10]
for bin_set in set_bin:
    counter = 0
    #make empty df to load indel with bin size 
    indel_df = pd.DataFrame({"pos": list(range(1,7601))})
    indel_df["DEL"] = [0] * 7600
    indel_df["INS"] = [0] * 7600
    indel_fname =  "indel_df/" + stage + "_" + rDNA + "_bin_" + str(bin_set) + ".tsv" 
    with open(fname , "r") as samfile:
        for _ in range(4):
            next(samfile)
        for line in samfile:    
            sam_content = line.split("\t")
            flag = sam_content[1]
            if flag != 4 :
                counter += 1
                pos = int(sam_content[3])
                cigar = str(sam_content[5])
                Read_45SrDNA_Cigar2df(pos, cigar , bin_set)
                sys.stdout.write('\r'+"{} cigar processed".format(counter))
        indel_df.to_csv(indel_fname, index=False, header=True, sep="\t")
    print("\nExtracted {} records of cigar to file with minimal INDEL size {}\n".format(counter , bin_set))

15585 cigar processed
Extracted 15585 records of cigar to file with minimal INDEL size 2

15585 cigar processed
Extracted 15585 records of cigar to file with minimal INDEL size 5

15585 cigar processed
Extracted 15585 records of cigar to file with minimal INDEL size 10



# move to Rstudio for plotting  


In [5]:
%%bash
samtools view -h bam/N2EM_45S_sort.bam > sam/N2EM_45S.sam
samtools view -h bam/N2L1_45S_sort.bam > sam/N2L1_45S.sam
samtools view -h bam/N2YA_45S_sort.bam > sam/N2YA_45S.sam
samtools view -h bam/AF16_45S_sort.bam > sam/AF16_45S.sam
samtools view -h bam/CB4856_45S_sort.bam > sam/CB4856_45S.sam